# Use vector data as a third spatial scale

Data from [Estimation of spatial distribution of irrigated crop areas in Europe for large-scale modelling applications](https://www.sciencedirect.com/science/article/pii/S0378377422000749?via%3Dihub#sec0060), via [Agri4Cast](https://agri4cast.jrc.ec.europa.eu/DataPortal/Index.aspx?o=).

Data also [available here](https://files.brightway.dev/europe_irrigated.gpkg).

In [ ]:
import bw2data as bd
import bw2regional as bwr
import time

In [ ]:
bd.projects.set_current("LC IMPACT case study")

In [ ]:
fp = "/srv/data/vectors/europe_irrigated.gpkg"

In [ ]:
bwr.geocollections["jrc-irrigation-wheat"] = {
    "filepath": fp,
    "field": "USCIE_GRID",        
}

In [ ]:
to_intersect = [
    "world",
    "watersheds-hh",
    "watersheds-eq-sw-certain",
    "watersheds-eq-sw-all",
    "particulate-matter",
    "ecoregions",
]

In [ ]:
jobs = []

for gc in to_intersect:
    jobs.append(bwr.remote.calculate_intersection(gc, "jrc-irrigation-wheat"))
    
while not all(job.status != 'started' for job in jobs):
    time.sleep(2)

In [ ]:
for gc in to_intersect:
    bwr.remote.intersection(gc, "jrc-irrigation-wheat")

In [ ]:
xt = bwr.ExtensionTable("jrc-irrigation-wheat")
xt.register(
    geocollection="jrc-irrigation-wheat",
    xt_field="IR_cereals"
)

In [ ]:
xt.import_from_map(mask=-999)

In [ ]:
my_method = ('LC-IMPACT',
  'Water Use',
  'Ecosystem Quality',
  'Surface Water',
  'Marginal',
  'All')


In [ ]:
lca = bwr.ExtensionTablesLCA(
    demand={('meal', 'meal'): 1},
    method=my_method,
    xtable="jrc-irrigation-wheat"
)
lca.lci()
lca.lcia()

In [ ]:
lca.score

In [ ]:
lca.geodataframe_ia_spatial_scale(cutoff=1e-5).explore(column='score_rel')